# YANDEX ML

In [175]:
import pandas as pd
import numpy as np
import pymorphy2
from  TextPreprocessing import TextPreprocessing
import pickle
from  profiler import Profiler

In [176]:
df = pd.read_csv('train.tsv',
                         names=['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply',
                                'label', 'confidence'],
                        header=None, sep='\t')

In [177]:

df.head()


,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,good,0.8753516175
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",neutral,0.9009682113
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",bad,0.8843202145
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,good,0.9825304673
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,good,0.8380535096


In [178]:
df.count()

context_id    93349
context_2     46605
context_1     69545
context_0     93349
reply_id      93349
reply         93344
label         93328
confidence    93289
dtype: int64

In [179]:
df = df[df['reply'].notnull()]

In [180]:
df_sparce = []
col_name = ['context_0','context_1','context_2']
cont_indexes = [
    pd.isnull(df['context_2']) & pd.isnull(df['context_1']),
    pd.isnull(df['context_2']) & pd.notnull(df['context_1']),
    pd.notnull(df['context_2']) & pd.notnull(df['context_1'])
]
# Один контекст
df_sparce.append(df[cont_indexes[0]])

# Два контекста
df_sparce.append(df[cont_indexes[1]])

# Три контекста
df_sparce.append(df[cont_indexes[2]])

In [188]:
df[cont_indexes[0]]

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
49,289676265247,NaN,NaN,мое имя дали мне родители .,0,теперь вам дают и имена .,good,0.8019585199
50,289676265247,NaN,NaN,мое имя дали мне родители .,1,полное имя ?,bad,0.9387921268
51,289676265247,NaN,NaN,мое имя дали мне родители .,2,какое имя ?,good,0.7983988626
52,289676265247,NaN,NaN,мое имя дали мне родители .,3,а твое имя - невер .,good,0.9076912001
53,289676265247,NaN,NaN,мое имя дали мне родители .,4,а мое ханна грейс .,good,0.9912839575
54,289676265247,NaN,NaN,мое имя дали мне родители .,5,какое ещё имя ?,good,0.5083296677
67,306950681470,NaN,NaN,почему бы тебе не развестись ?,0,потому что я не хочу в тюрьму .,good,0.8459445168
68,306950681470,NaN,NaN,почему бы тебе не развестись ?,1,потому что я боюсь .,good,0.9248947068
69,306950681470,NaN,NaN,почему бы тебе не развестись ?,2,что ты хочешь ?,good,0.5111830151
70,306950681470,NaN,NaN,почему бы тебе не развестись ?,3,"нет , мне не стоит .",good,0.8956661385


In [181]:
tp = TextPreprocessing()
col_amount = 0

'''
data_sparced - Список датафреймов
len(data_sparced) = 3
Каждый элемент списка - подвыборка исходного датафрейма
data_sparced[0] - подвыбока, содержащая только один контекст
data_sparced[1] - подвыбока, содержащая два контекста
data_sparced[2] - подвыбока, содержащая три контекста
'''
data_sparced = []
data_sparced_clear = []
with Profiler():
    for df_part in df_sparce:
        data_sparced.append(df[cont_indexes[col_amount]])
        data_sparced_clear.append(df[cont_indexes[col_amount]])
        for i in range(col_amount+1):
            prep_col_clear = df_part[col_name[i]].map(tp.preprocess_sentence).map(tp.list_to_text)
            data_sparced_clear[col_amount][col_name[i]] = prep_col_clear
            data_sparced_clear[col_amount]['reply'] = df[cont_indexes[col_amount]]['reply'].map(tp.preprocess_sentence).map(tp.list_to_text)
        col_amount+=1

C:\Users\kilim\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\kilim\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Затраченное время: 204.9233431816101


In [186]:
data_sparced_clear[0][:100]

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
49,289676265247,NaN,NaN,имя дать родитель,0,давать имя,good,0.8019585199
50,289676265247,NaN,NaN,имя дать родитель,1,полный имя,bad,0.9387921268
51,289676265247,NaN,NaN,имя дать родитель,2,имя,good,0.7983988626
52,289676265247,NaN,NaN,имя дать родитель,3,твой имя невера,good,0.9076912001
53,289676265247,NaN,NaN,имя дать родитель,4,,good,0.9912839575
54,289676265247,NaN,NaN,имя дать родитель,5,еще имя,good,0.5083296677
67,306950681470,NaN,NaN,почему развестись,0,хотеть тюрьма,good,0.8459445168
68,306950681470,NaN,NaN,почему развестись,1,бояться,good,0.9248947068
69,306950681470,NaN,NaN,почему развестись,2,хотеть,good,0.5111830151
70,306950681470,NaN,NaN,почему развестись,3,стоить,good,0.8956661385


In [183]:
df[cont_indexes[2]].context_1

0                                        о , я тебя вижу .
1                                        о , я тебя вижу .
2                                        о , я тебя вижу .
3                                        о , я тебя вижу .
4                                        о , я тебя вижу .
5                                        о , я тебя вижу .
6         слушайте , мы с женой . . . совсем заблудились .
7         слушайте , мы с женой . . . совсем заблудились .
8         слушайте , мы с женой . . . совсем заблудились .
9         слушайте , мы с женой . . . совсем заблудились .
10        слушайте , мы с женой . . . совсем заблудились .
11        слушайте , мы с женой . . . совсем заблудились .
12                        кажется , я там ни разу не был .
13                        кажется , я там ни разу не был .
14                        кажется , я там ни разу не был .
15                        кажется , я там ни разу не был .
16                        кажется , я там ни разу не был

In [184]:
data_sparced[1][]

SyntaxError: invalid syntax (<ipython-input-184-173547f59811>, line 1)

In [185]:
data_sparced[1].head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
61,300113641606,NaN,не это .,"хотя , какого <CENSORED> ?",0,тише .,good,0.9291954342
62,300113641606,NaN,не это .,"хотя , какого <CENSORED> ?",1,я в <CENSORED> .,bad,0.9949661191
63,300113641606,NaN,не это .,"хотя , какого <CENSORED> ?",2,наверное это не важно .,good,0.7296121132
64,300113641606,NaN,не это .,"хотя , какого <CENSORED> ?",3,"все в порядке , женщина .",good,0.8377676781
65,300113641606,NaN,не это .,"хотя , какого <CENSORED> ?",4,что ?,neutral,0.9838724208


In [187]:
with Profiler():
    with open("canonized_clear.pickle",'wb') as file:
        pickle.dump(data_sparced_clear, file)
        
with Profiler():
    with open("canonized_marked.pickle",'wb') as file:
        pickle.dump(data_sparced, file)
'''
Пример загрузки файла из пикла
        
with open("kfg",'rb') as file:
    asd = pickle.load(file)
'''


Затраченное время: 0.2579798698425293
Затраченное время: 0.12314796447753906


'\nПример загрузки файла из пикла\n        \nwith open("kfg",\'rb\') as file:\n    asd = pickle.load(file)\n'